# Test if MendelImpute can impute untyped SNPs

In [1]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using StatsBase

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /Users/biona001/.julia/packages/Revise/439di/src/Revise.jl:1108
┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1273


## Generate subset of markers for prephasing

In [5]:
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function filter_and_mask()
    for chr in [22]
        # filter chromosome data for unique snps
        data = "../raw/ALL.chr$chr.phase3_v5.shapeit2_mvncall_integrated.noSingleton.genotypes.vcf.gz"
        full_record_index = .!find_duplicate_marker(data)
        @time VCFTools.filter(data, full_record_index, 1:nsamples(data), 
            des = "chr$chr.uniqueSNPs.vcf.gz")
    
        # generate target file with 250 samples and 100k snps
        n = 250
        p = 100000
        total_snps = nrecords("chr$chr.uniqueSNPs.vcf.gz")
        Random.seed!(2020)
        record_idx = falses(total_snps)
        record_idx[1:p] .= true
        shuffle!(record_idx)
        sample_idx = falses(nsamples("chr$chr.uniqueSNPs.vcf.gz"))
        sample_idx[1:n] .= true
        shuffle!(sample_idx)
        @time VCFTools.filter("chr$chr.uniqueSNPs.vcf.gz", record_idx, sample_idx, 
            des = "target.chr$chr.prephase.vcf.gz")

        # generate target panel with all snps
        @time VCFTools.filter("chr$chr.uniqueSNPs.vcf.gz", 
            1:total_snps, sample_idx, des = "target.chr$chr.full.vcf.gz")
        
        # also generate reference panel without target samples
        @time VCFTools.filter("chr$chr.uniqueSNPs.vcf.gz", 
            1:total_snps, .!sample_idx, des = "ref.chr$chr.excludeTarget.vcf.gz")
        
        # unphase and mask 1% entries in target file
        masks = falses(p, n)
        missingprop = 0.01
        Random.seed!(2020)
        for j in 1:n, i in 1:p
            rand() < missingprop && (masks[i, j] = true)
        end
        @time mask_gt("target.chr$chr.prephase.vcf.gz", masks, 
            des="target.chr$chr.prephase.masked.vcf.gz", unphase=true)
    
        # generate subset of reference file that matches target file
        @time conformgt_by_pos("ref.chr$chr.excludeTarget.vcf.gz", 
            "target.chr$(chr).prephase.masked.vcf.gz", 
            "chr$chr.aligned", "$chr", 1:typemax(Int))
        if nrecords("chr$chr.aligned.tgt.vcf.gz") == p
            rm("chr$chr.aligned.tgt.vcf.gz", force=true) # perfect match
        else
            error("target file has SNPs not matching in reference file! Shouldn't happen!")
        end
        mv("chr22.aligned.ref.vcf.gz", "ref.chr22.aligned.vcf.gz")
    end 
end
@time filter_and_mask()

547.645319 seconds (4.91 G allocations: 463.567 GiB, 6.91% gc time)
433.979604 seconds (4.98 G allocations: 468.237 GiB, 10.20% gc time)
463.505887 seconds (5.67 G allocations: 523.322 GiB, 10.87% gc time)
1022.502225 seconds (12.13 G allocations: 936.019 GiB, 12.45% gc time)
 13.274375 seconds (76.61 M allocations: 8.666 GiB, 6.78% gc time)


SystemError: SystemError: opening file "target.chr22.masked.vcf.gz": No such file or directory

In [7]:
chr = 22
p = 100000
@time conformgt_by_pos("ref.chr$chr.excludeTarget.vcf.gz", 
    "target.chr$(chr).prephase.masked.vcf.gz", 
    "chr$chr.aligned", "$chr", 1:typemax(Int))
if nrecords("chr$chr.aligned.tgt.vcf.gz") == p
    rm("chr$chr.aligned.tgt.vcf.gz", force=true) # perfect match
else
    error("target file has SNPs not matching in reference file! Shouldn't happen!")
end

┌ Info: Match target POS to reference POS
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:172
Progress: 100%|█████████████████████████████████████████| Time: 0:12:23


769.477247 seconds (7.23 G allocations: 672.427 GiB, 13.24% gc time)


┌ Info: 100000 records are matched
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:239


### Missing rate

In typed markers, 1% of data is missing at random. In addition, only 84% of all markers are not typed (i.e. systematically missing). 

In [3]:
tgtfile = "target.chr22.prephase.masked.vcf.gz"
reffile = "ref.chr22.excludeTarget.vcf.gz"
missing_rate = 1 - nrecords(tgtfile) / nrecords(reffile)

0.844946576342879

# MendelImpute on typed and untyped markers with dp

In [4]:
Threads.nthreads()

8

In [11]:
tgtfile = "target.chr22.prephase.masked.vcf.gz"
reffile = "ref.chr22.excludeTarget.vcf.gz"
reffile_aligned = "ref.chr22.aligned.vcf.gz"
X_prephase_complete = "target.chr22.prephase.vcf.gz"
X_full_complete = "target.chr22.full.vcf.gz"
@show nrecords(tgtfile), nsamples(tgtfile)
@show nrecords(reffile), nsamples(reffile)
@show nrecords(reffile_aligned), nsamples(reffile_aligned)
@show nrecords(X_prephase_complete), nsamples(X_prephase_complete)
@show nrecords(X_full_complete), nsamples(X_full_complete)

(nrecords(tgtfile), nsamples(tgtfile)) = (100000, 250)
(nrecords(reffile), nsamples(reffile)) = (644939, 2254)
(nrecords(reffile_aligned), nsamples(reffile_aligned)) = (100000, 2254)
(nrecords(X_prephase_complete), nsamples(X_prephase_complete)) = (100000, 250)
(nrecords(X_full_complete), nsamples(X_full_complete)) = (644939, 250)


(644939, 250)

### Error rate on typed snps 

In [5]:
# using incomplete ref panel
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
Random.seed!(2020)
function run()
    X_complete = convert_gt(Float32, "target.chr22.prephase.vcf.gz")
#     X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
    n, p = size(X_complete)
    chr = 22
    for width in [500, 1000]
        println("Imputing typed SNPs only with dynamic programming, width = $width")
        tgtfile = "target.chr$chr.prephase.masked.vcf.gz"
        reffile = "ref.chr$chr.excludeTarget.vcf.gz"
        outfile = "mendel.imputed.dp$width.vcf.gz"
        reffile_aligned = "ref.chr$chr.aligned.vcf.gz"
        @time phase(tgtfile, reffile, reffile_aligned = reffile_aligned,
            outfile = outfile, width = width, fast_method=false)
        X_mendel = convert_gt(Float32, outfile)
        println("error on typed data = $(sum(X_mendel .!= X_complete) / n / p) \n")
    end
end
run()

Imputing typed SNPs only with dynamic programming, width = 500
Running chunk 1 / 1


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:00:27
Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:36
Imputing samples...100%|████████████████████████████████| Time: 0:02:29
Writing to file...100%|█████████████████████████████████| Time: 0:00:10


296.258943 seconds (635.06 M allocations: 67.488 GiB, 2.70% gc time)
error on typed data = 0.00013432
Imputing typed SNPs only with dynamic programming, width = 1000
Running chunk 1 / 1


Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Importing reference haplotype files...100%|█████████████| Time: 0:00:31
Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:21
Imputing samples...100%|████████████████████████████████| Time: 0:06:10
Writing to file...100%|█████████████████████████████████| Time: 0:00:09


508.449093 seconds (634.08 M allocations: 65.385 GiB, 1.60% gc time)
error on typed data = 0.00017911999999999998


### Error rate on untyped + typed snps

In [4]:
# dp method using incomplete ref panel
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
Random.seed!(2020)
function run()
#     X_complete = convert_gt(Float32, "target.chr22.prephase.vcf.gz")
    X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
    n, p = size(X_complete)
    chr = 22
    for width in [500, 1000]
        println("Imputing typed + untyped SNPs with dynamic programming, width = $width")
        tgtfile = "target.chr$chr.prephase.masked.vcf.gz"
        reffile = "ref.chr$chr.excludeTarget.vcf.gz"
        outfile = "mendel.imputed.dp$width.vcf.gz"
        reffile_aligned = "ref.chr$chr.aligned.vcf.gz"
        @time phase(tgtfile, reffile, reffile_aligned = reffile_aligned, impute=true, 
            outfile = outfile, width = width, fast_method=false)
        X_mendel = convert_gt(Float32, outfile)
        println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")
    end
end
run()

Imputing typed SNPs only with dynamic programming, width = 500
Running chunk 1 / 1


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:00:30
Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:56
Imputing samples...100%|████████████████████████████████| Time: 0:02:31
Writing to file...100%|█████████████████████████████████| Time: 0:06:33


913.189622 seconds (8.69 G allocations: 806.302 GiB, 10.39% gc time)
error overall = 0.011392010717292643
Imputing typed SNPs only with dynamic programming, width = 1000
Running chunk 1 / 1


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:00:27
Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:14
Imputing samples...100%|████████████████████████████████| Time: 0:06:04
Writing to file...100%|█████████████████████████████████| Time: 0:06:21


1068.349860 seconds (8.67 G allocations: 803.618 GiB, 8.57% gc time)
error overall = 0.015489930055400588


# MendelImpute on typed/untyped markers with fast method

In [14]:
# fast method using incomplete ref panel
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
Random.seed!(2020)
function run()
#     X_complete = convert_gt(Float32, "target.chr22.prephase.vcf.gz")
    X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
    n, p = size(X_complete)
    chr = 22
    for width in 100:100:500
        println("Imputing typed + untyped SNPs with dynamic programming, width = $width")
        tgtfile = "target.chr$chr.prephase.masked.vcf.gz"
        reffile = "ref.chr$chr.excludeTarget.vcf.gz"
        outfile = "mendel.imputed.dp$width.vcf.gz"
        reffile_aligned = "ref.chr$chr.aligned.vcf.gz"
        @time phase(tgtfile, reffile, reffile_aligned = reffile_aligned, impute=true, 
            outfile = outfile, width = width, fast_method=true)
        X_mendel = convert_gt(Float32, outfile)
        println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")
    end
end
run()

Imputing typed + untyped SNPs with dynamic programming, width = 100
Running chunk 1 / 1


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:00:27
Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:28
Writing to file...100%|█████████████████████████████████| Time: 0:06:44


689.956454 seconds (8.67 G allocations: 798.973 GiB, 14.26% gc time)
error overall = 0.010857702821507149 

Imputing typed + untyped SNPs with dynamic programming, width = 200
Running chunk 1 / 1


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:00:27
Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:09
Merging breakpoints...100%|█████████████████████████████| Time: 0:00:14
Writing to file...100%|█████████████████████████████████| Time: 0:06:48


740.672125 seconds (8.67 G allocations: 801.593 GiB, 12.78% gc time)
error overall = 0.010387438191828994 

Imputing typed + untyped SNPs with dynamic programming, width = 300
Running chunk 1 / 1


...

816.463186 seconds (8.67 G allocations: 803.592 GiB, 11.90% gc time)
error overall = 0.010579456351685973 

Imputing typed + untyped SNPs with dynamic programming, width = 400
Running chunk 1 / 1


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:00:29
Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:37
Writing to file...100%|█████████████████████████████████| Time: 0:06:30


790.604516 seconds (8.67 G allocations: 804.614 GiB, 11.56% gc time)
error overall = 0.011033117860758924 

Imputing typed + untyped SNPs with dynamic programming, width = 500
Running chunk 1 / 1


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:00:27
Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:34
Merging breakpoints...100%|█████████████████████████████| Time: 0:01:19
Writing to file...100%|█████████████████████████████████| Time: 0:06:50


854.401966 seconds (8.67 G allocations: 805.029 GiB, 11.55% gc time)
error overall = 0.011707159901944215 



# Complete ref panel gives good error % for typed snps

In [13]:
# complete ref panel, includes time to generate ref.aligned
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
Random.seed!(2020)
function run()
    X_complete = convert_gt(Float32, "target.chr22.prephase.vcf.gz")
#     X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
    n, p = size(X_complete)
    chr = 22
    for width in [500, 1000]
        println("Imputing typed SNPs only with dynamic programming, width = $width")
        tgtfile = "target.chr$chr.prephase.masked.vcf.gz"
        reffile = "chr$chr.uniqueSNPs.vcf.gz"
        outfile = "mendel.imputed.dp$width.vcf.gz"
        @time phase(tgtfile, reffile, outfile = outfile, width = width, fast_method=false)
        X_mendel = convert_gt(Float32, outfile)
        println("error on prephase data = $(sum(X_mendel .!= X_complete) / n / p) \n")
    end
end
run()

┌ Info: Match target POS to reference POS
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:172
Progress: 100%|█████████████████████████████████████████| Time: 0:13:31Imputing samples... 89%|████████████████████████████▌   |  ETA: 0:01:38
┌ Info: 100000 records are matched
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:239
Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:00:34
Computing optimal haplotype pairs...100%|███████████████| Time: 0:02:13
Imputing samples...100%|████████████████████████████████| Time: 0:00:07
Writing to file...100%|█████████████████████████████████| Time: 0:00:11
┌ Info: Match target POS to reference POS
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:172
Progress: 100%|█████████████████████████████████████████| Time: 0:10:00
┌ Info: 100000 records are matched
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformg

Imputing typed SNPs only with dynamic programming, width = 500
Running chunk 1 / 1
1059.540364 seconds (8.67 G allocations: 808.684 GiB, 7.92% gc time)
error on prephase data = 5.7199999999999994e-6 

Imputing typed SNPs only with dynamic programming, width = 1000
Running chunk 1 / 1
823.782148 seconds (8.67 G allocations: 806.468 GiB, 12.22% gc time)
error on prephase data = 5.6e-7 



In [14]:
# complete ref panel
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
Random.seed!(2020)
function run()
#     X_complete = convert_gt(Float32, "target.chr22.prephase.vcf.gz")
    X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
    n, p = size(X_complete)
    chr = 22
    for width in [500, 1000, 2000]
        println("Imputing typed + untyped SNPs with dynamic programming, width = $width")
        tgtfile = "target.chr$chr.prephase.masked.vcf.gz"
        reffile = "chr$chr.uniqueSNPs.vcf.gz"
        outfile = "mendel.imputed.dp$width.vcf.gz"
        @time phase(tgtfile, reffile, impute=true, outfile = outfile, 
            width = width, fast_method=false)
        X_mendel = convert_gt(Float32, outfile)
        println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")
    end
end
run()

┌ Info: Match target POS to reference POS
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:172
Progress: 100%|█████████████████████████████████████████| Time: 0:09:09
┌ Info: 100000 records are matched
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:239
Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:00:30
Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:51
Imputing samples...100%|████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:07:20
┌ Info: Match target POS to reference POS
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:172
Progress: 100%|█████████████████████████████████████████| Time: 0:09:28
┌ Info: 100000 records are matched
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:239
Importing genotype file...100%|█████████████████████████| Time

Imputing typed + untyped SNPs with dynamic programming, width = 500
Running chunk 1 / 1
1428.346320 seconds (17.51 G allocations: 1.576 TiB, 13.55% gc time)
error overall = 0.0002901545727580438 

Imputing typed + untyped SNPs with dynamic programming, width = 1000
Running chunk 1 / 1
1464.793405 seconds (17.51 G allocations: 1.573 TiB, 13.52% gc time)
error overall = 1.1852283704350334e-5 

Imputing typed + untyped SNPs with dynamic programming, width = 2000
Running chunk 1 / 1
1399.438588 seconds (17.51 G allocations: 1.571 TiB, 13.52% gc time)
error overall = 1.8606410838854526e-8 



# Beagle 5.0 on complete ref panel

In [9]:
# rename all samples in tgtfile to tmp_i, store result in tgt_tmp
chr = 22
tgtfile = "target.chr$chr.prephase.masked.vcf.gz"
tgt_tmp = "target.chr$chr.prephase.masked.tmp.vcf.gz"
reader = VCF.Reader(openvcf(tgtfile, "r"))
metainfo = VCF.header(reader).metainfo
sampleID = VCF.header(reader).sampleID
[sampleID[i] = "tmp$i" for i in 1:length(sampleID)]

#output tmp target 
writer = VCF.Writer(openvcf(tgt_tmp, "w"), VCF.Header(metainfo, sampleID))
[VCF.write(writer, record) for record in reader]
flush(writer); close(reader); close(writer)

In [11]:
# beagle 5
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function beagle()
    chr = 22
    tgtfile = "target.chr$chr.prephase.masked.tmp.vcf.gz"
    reffile = "chr$chr.uniqueSNPs.vcf.gz"
    outfile = "beagle.imputed.completeREF"
    Base.run(`java -Xmx15g -jar beagle.28Sep18.793.jar gt=$tgtfile ref=$reffile out=$outfile nthreads=4`)

    # beagle error rate    
    X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
    X_beagle = convert_gt(Float32, "beagle.imputed.completeREF.vcf.gz")
    n, p = size(X_complete)
    println("error overall = $(sum(X_beagle .!= X_complete) / n / p) \n")
end
beagle()

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 03:16 PM PDT on 23 Apr 2020

Command line: java -Xmx13653m -jar beagle.28Sep18.793.jar
  gt=target.chr22.prephase.masked.tmp.vcf.gz
  ref=chr22.uniqueSNPs.vcf.gz
  out=beagle.imputed.completeREF
  nthreads=4

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,504
Study samples:             250

Window 1 (22:16050115-51244237)
Reference markers:     644,939
Study markers:         100,000

Burnin  iteration 1:           41 seconds
Burnin  iteration 2:           47 seconds
Burnin  iteration 3:           49 seconds
Burnin  iteration 4:           46 seconds
Burnin  iteration 5:           51 seconds
Burnin  iteration 6:           52 seconds

Phasing iteration 1:           50 seconds
Phasing iteration 2:           49 seconds
Phasing iteration 3:           48 seconds
Phasing iteration 4:           53 seconds
Ph

# Beagle 5.0 on incomplete ref panel

In [4]:
# beagle 5
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function beagle()
    chr = 22
    tgtfile = "target.chr$chr.prephase.masked.vcf.gz"
    reffile = "ref.chr$chr.excludeTarget.vcf.gz"
    outfile = "beagle.imputed"
    run(`java -Xmx15g -jar beagle.28Sep18.793.jar gt=$tgtfile ref=$reffile out=$outfile nthreads=4`)
end
beagle()

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 02:06 PM PDT on 23 Apr 2020

Command line: java -Xmx13653m -jar beagle.28Sep18.793.jar
  gt=target.chr22.prephase.masked.vcf.gz
  ref=ref.chr22.excludeTarget.vcf.gz
  out=beagle.imputed
  nthreads=4

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,254
Study samples:             250

Window 1 (22:16050115-51244237)
Reference markers:     644,939
Study markers:         100,000

Burnin  iteration 1:           45 seconds
Burnin  iteration 2:           49 seconds
Burnin  iteration 3:           48 seconds
Burnin  iteration 4:           50 seconds
Burnin  iteration 5:           56 seconds
Burnin  iteration 6:           59 seconds

Phasing iteration 1:           54 seconds
Phasing iteration 2:           58 seconds
Phasing iteration 3:           57 seconds
Phasing iteration 4:           54 seconds
Phasing ite

ArgumentError: ArgumentError: vcffile name should end with vcf or vcf.gz

In [5]:
# beagle error rate    
X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
X_beagle = convert_gt(Float32, "beagle.imputed.vcf.gz")
n, p = size(X_complete)
println("error overall = $(sum(X_beagle .!= X_complete) / n / p) \n")

error overall = 0.006312125642890258 



# Eagle 2 + Minimac4

In order to use the reference panel in Eagle 2's prephase option, one must first convert it to `.bcf` format via e.g. `htslib` which is *extremely* difficult to install. Even after we went through all the hard work to obtain the final `.bcf` reference file (see commands below), eagle 2.4 STILL SAYS the file is not acceptable (not bgzipped or some processing error). Therefore, I have no choice but to prephase without the reference panel. 

In [ ]:
# THIS CHUNK OF CODE DID NOT WORK
module load htslib/1.9                   # load htslib on hoffman
gunzip ref.chr22.excludeTarget.vcf.gz    # unzip .vcf.gz
bgzip ref.chr22.excludeTarget.vcf        # bgzip the vcf file
bcftools index -f ref.chr22.excludeTarget.vcf.gz   # load some required index, not sure if needed
bcftools view ref.chr22.excludeTarget.vcf > ref.chr22.excludeTarget.bcf # create .bcf -> CREATED FILE BUT says not bgzipped
bgzip ref.chr22.excludeTarget.bcf       # try to bgzip .bcf file -> cannot do so

In [ ]:
# run eagle 2.4: 566.355 sec on amd-2431 machine (can only run on linux systems)
eagle --vcf=target.chr22.prephase.masked.vcf.gz --outPrefix=eagle.phased --numThreads=4 --geneticMapFile=../Eagle_v2.4.1/tables/genetic_map_hg19_withX.txt.gz

In [ ]:
# convert ref file to m3vcf format (Total Run completed in 1 hours, 46 mins, 24 seconds)
/u/home/b/biona001/haplotype_comparisons/Minimac3/bin/Minimac3 --refHaps ref.chr22.excludeTarget.vcf.gz --processReference --prefix ref.chr22.excludeTarget

In [ ]:
# run minimac4 (43 mins, 16 seconds)
minimac4 --refHaps ref.chr22.excludeTarget.m3vcf.gz --haps eagle.phased.vcf.gz --prefix minimac.imputed.chr22 --format GT --cpus 8

In [4]:
# minimac4 error rate    
X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
X_minimac = convert_gt(Float32, "minimac.imputed.chr22.dose.vcf.gz")
n, p = size(X_complete)
println("error overall = $(sum(X_minimac .!= X_complete) / n / p) \n")

error overall = 0.007229769016914778 

